# Biomedical Image Analysis in Python
The field of biomedical imaging has exploded in recent years - but for the uninitiated, even loading data can be a challenge! In this introductory course, you'll learn the fundamentals of image analysis using NumPy, SciPy, and Matplotlib. You'll navigate through a whole-body CT scan, segment a cardiac MRI time series, and determine whether Alzheimer’s disease changes brain structure. Even if you have never worked with images before, you will finish the course with a solid toolkit for entering this dynamic field.

**Instructor:** Stephen Bailey is lead data scientist at Immuta, where he researches and develops best practices in data governance. He is passionate about democratizing science and is an enthusiastic member of the Python community. Stephen has published a number of papers on leveraging medical imaging technology to better understand health and disease, and he is excited to introduce others to this dynamic research field!

# $\star$ Chapter 1: Exploration
Prepare to conquer the Nth dimension! To begin the course, you'll learn how to load, build and navigate N-dimensional images using a CT image of the human chest. You'll also leverage the useful ImageIO package and hone your NumPy and matplotlib skills.

### Image data
   Since the first X-ray in 1895, medical imaging technology has advanced clinical care and opened up new fields of scientific investigation. The amount of imaging data is exploding: by 2020, there is estimated to be more than 3.5 billion terabytes of imaging data in the U.S. alone. This has created amazing opportunities for analysis: 
   * Measuring organ shape and size
   * Creating detailed reconstructions of anatomy
   * Analyzing tissue composition
   * Predicting pathology
   * Much, much more...
   The complexity and variety of the data can make it an intimidating field to jump into. Fortunately, there are fantastic tools and a wealth of resources to support you. 
  
#### Toolbox
* `ImageIO`
* `NumPy`
* `SciPy`
* `matplotlib`

#### Course Objectives

* **1) Exploration**
    * Loading images
    * N-D data
    * Subplots
    
* **2) Masks and Filters**
    * Intensity distributions
    * Convolutions
    * Edge detection
    
* **3) Measurement**
    * Labelling
    * Multi-object measurement
    * Morphology
    
* **4) Image Comparison**
    * Transformations
    * Resampling
    * Cost functions
    * Normalization
    
* While this only scratches the surface of the field, it will give you a firm foundation for advanced concepts like classification and segmentation.

### Loading images

#### `imageio`
* read and save images with `imread`

```
import imagio
im = imagio.imread('body-001.dcm')
```
   * One useful feature of `imageio` is that it can read **DICOM files, the standard format for human medical imaging.**
   * The data is read in as an **image object**, which is a type of NumPy array.
* To access specific values from your image, you can slice out a single value or a range of index values along each dimension
    * Slice the array by specifying values along each available dimension
    
```
im[0, 0]
im[0:2, 0:2]
```

### Metadata
* Images are always acquired in a specific context; this information is often referred to as **metadata**.
* **Metadata:** the who, what, when, where, and how of image acquisition
* `imagio` loads available metadata into a dictionary, accessible through the `meta` attribute
* **`image.meta`**
* This is especially important for **DICOM** files, which contain a lot of patient and acquisition information
* Since `meta` is a Python dictionary, you can access specific information by indexing it with one of the available keys.
* **You can also call the `keys()` method to list all of the available metadata.**
    * `image.meta.keys()`
    
#### Plotting images
* Visualization is fundamental to image analysis, and we'll rely heavily on matplotlib and its `imshow()` function throughout this course.


In [2]:
import numpy as np
import scipy
import imageio
import matplotlib.pyplot as plt
from pprint import pprint

plt.rcParams['figure.figsize'] = (10, 8)

* `plt.imshow(im, cmap='gray')
* If the image does not have color, add `cmap='gray'` to plot it in grayscale
* Many colormaps available but often shown in grayscale (`cmap='gray')
* **To reduce clutter, it's common to turn off the axis ticks, labels, and frame with `plt.axis('off')`**
* Finally, call `plt.show()` per usual to render the image. 

#### Loading Images Exercises
In this chapter, we'll work with sections of a [computed tomography (CT) scan](https://en.wikipedia.org/wiki/CT_scan) from [The Cancer Imaging Archive](https://www.cancerimagingarchive.net/about-the-cancer-imaging-archive-tcia/) . CT uses a rotating X-ray tube to create a 3D image of the target area.

The actual content of the image depends on the instrument used: photographs measure visible light, x-ray and CT measure radiation absorbance, and MRI scanners measure magnetic fields.

To warm up, use the `imageio` package to load a single DICOM image from the scan volume and check out a few of its attributes.

```
# Import ImageIO
import imageio

# Load "chest-220.dcm"
im = imread('chest-220.dcm')

# Print image attributes
print('Image type:', type(im))
print('Shape of image array:', im.shape)
```

`imageio` is a versatile package. It can read in a variety of image data, including JPEG, PNG, and TIFF. But it's especially useful for its ability to handle DICOM files.